<a href="https://colab.research.google.com/github/tryfuIhaIckme/translators/blob/main/YandexML_Spring_22_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Код для обучения новой модели

In [ ]:
# -*- coding: utf-8 -*-
# Блок 1: Установка и импорт библиотек
# (Код остается таким же, как в предыдущем ответе)
# -------------------------------------
# Убедитесь, что у вас установлены необходимые библиотеки.
# Если нет, раскомментируйте и выполните следующую строку в ячейке Jupyter Notebook:
# !pip install torch torchvision torchaudio pandas numpy scikit-learn matplotlib tqdm

import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm  # Для красивых индикаторов прогресса в Jupyter
import time # Для замера времени

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

print(f"PyTorch version: {torch.__version__}")

# Проверка доступности GPU
if torch.cuda.is_available():
    print("GPU доступен:", torch.cuda.get_device_name(0))
else:
    print("GPU недоступен, используется CPU.")

# Определение текущего времени для возможного использования в именах файлов
# Используем UTC для большей универсальности
current_time_str = time.strftime("%Y%m%d_%H%M%S_UTC", time.gmtime())
print(f"Текущее время UTC: {current_time_str}")

PyTorch version: 2.6.0+cu124
GPU недоступен, используется CPU.
Текущее время UTC: 20250408_125905_UTC


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Блок 2: Конфигурация
# (Код остается таким же, используем 'id' и 'category' для activity_categories.csv)
# --------------------
class Config:
    # !!! ВАЖНО: Укажите правильный путь к вашим данным !!!
    # Пример для Google Colab, если данные в /content/drive/MyDrive/kaggle/input/human_poses_data/
    DATA_PATH = '/content/drive/MyDrive/kaggle/input/human_poses_data/'
    # Если данные в другом месте, измените DATA_PATH

    TRAIN_IMG_DIR = os.path.join(DATA_PATH, 'img_train')
    TEST_IMG_DIR = os.path.join(DATA_PATH, 'img_test')
    TRAIN_CSV_PATH = os.path.join(DATA_PATH, 'train_answers.csv')
    CATEGORIES_CSV_PATH = os.path.join(DATA_PATH, 'activity_categories.csv')

    # Пути для сохранения результатов (в текущей папке блокнота)
    BEST_MODEL_WEIGHTS_PATH = f'/content/drive/MyDrive/kaggle/best_model_weights_{current_time_str}.pth'
    SUBMISSION_PATH = f'/content/drive/MyDrive/kaggle/submission_{current_time_str}.csv'

    # Параметры модели и обучения
    IMAGE_SIZE = 128
    BATCH_SIZE = 64
    NUM_EPOCHS = 100
    LEARNING_RATE = 1e-3
    VALIDATION_SPLIT = 0.15
    RANDOM_STATE = 42
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    NUM_WORKERS = 2

    # Параметры архитектуры
    NUM_CLASSES = None

    # Имена столбцов (будут проверены и, возможно, изменены в Блоке 3)
    TRAIN_ID_COLUMN = 'Id' # Предполагаемое имя столбца ID в train_answers.csv
    TRAIN_TARGET_COLUMN = 'target_feature' # Предполагаемое имя столбца меток в train_answers.csv
    TEST_ID_COLUMN = 'Id' # Предполагаемое имя столбца ID в test (для submission)

config = Config()

# Загрузка информации о категориях (используем 'id' и 'category')
category_map = None
try:
    activity_df = pd.read_csv(config.CATEGORIES_CSV_PATH)
    print(f"Файл {config.CATEGORIES_CSV_PATH} загружен. Столбцы:", activity_df.columns.tolist())
    if 'id' in activity_df.columns and 'category' in activity_df.columns:
        config.NUM_CLASSES = activity_df['id'].nunique()
        print(f"Количество классов: {config.NUM_CLASSES}")
        category_map = pd.Series(activity_df.category.values, index=activity_df.id).to_dict()
        print("Карта категорий создана:")
        # print(category_map) # Раскомментируйте, если хотите увидеть всю карту
    else:
        print(f"!!! Ошибка: В {config.CATEGORIES_CSV_PATH} не найдены столбцы 'id' или 'category'.")
        config.NUM_CLASSES = None
except FileNotFoundError:
    print(f"!!! Ошибка: Файл {config.CATEGORIES_CSV_PATH} не найден.")
    config.NUM_CLASSES = None
except Exception as e:
    print(f"!!! Произошла ошибка при загрузке категорий: {e}")
    config.NUM_CLASSES = None

# Установка seed
np.random.seed(config.RANDOM_STATE)
torch.manual_seed(config.RANDOM_STATE)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(config.RANDOM_STATE)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print(f"\nКонфигурация загружена.")
print(f"Устройство: {config.DEVICE}")
print(f"Количество классов: {config.NUM_CLASSES}")
print(f"Путь к данным: {config.DATA_PATH}")

In [ ]:
# Блок 3: Загрузка и подготовка данных (С ИЗМЕНЕНИЯМИ)
# -----------------------------------
# Добавляем диагностику и используем правильные имена столбцов.

# --- Диагностика train_answers.csv ---
print("\n--- Диагностика train_answers.csv ---")
try:
    temp_train_df = pd.read_csv(config.TRAIN_CSV_PATH, nrows=5)
    print(f"Первые 5 строк файла {config.TRAIN_CSV_PATH}:")
    print(temp_train_df)
    actual_columns = temp_train_df.columns.tolist()
    print("\nНазвания столбцов:")
    print(actual_columns)

    # Автоматически определяем имена столбцов, если они отличаются от дефолтных
    # Эвристика: первый столбец - ID, второй - target. Может потребовать ручной корректировки!
    if config.TRAIN_ID_COLUMN not in actual_columns and len(actual_columns) > 0:
        print(f"Предупреждение: Столбец '{config.TRAIN_ID_COLUMN}' не найден. Используем первый столбец '{actual_columns[0]}' как ID.")
        config.TRAIN_ID_COLUMN = actual_columns[0]
    if config.TRAIN_TARGET_COLUMN not in actual_columns and len(actual_columns) > 1:
         print(f"Предупреждение: Столбец '{config.TRAIN_TARGET_COLUMN}' не найден. Используем второй столбец '{actual_columns[1]}' как Target.")
         config.TRAIN_TARGET_COLUMN = actual_columns[1]
    elif len(actual_columns) <=1 and config.TRAIN_TARGET_COLUMN not in actual_columns :
         print(f"!!! Ошибка: Не найден столбец для меток ('{config.TRAIN_TARGET_COLUMN}' или второй столбец).")
         config.TRAIN_TARGET_COLUMN = None # Сбрасываем, чтобы показать проблему

    print(f"Используемые имена столбцов: ID='{config.TRAIN_ID_COLUMN}', Target='{config.TRAIN_TARGET_COLUMN}'")

except FileNotFoundError:
    print(f"!!! Ошибка: Файл {config.TRAIN_CSV_PATH} не найден.")
    actual_columns = []
except Exception as e:
    print(f"!!! Произошла ошибка при чтении {config.TRAIN_CSV_PATH}: {e}")
    actual_columns = []
# --- Конец диагностики ---


# Загрузка информации об обучающих данных (используем определенные имена столбцов)
train_df_full = None
try:
    # Теперь читаем весь файл
    train_df_full = pd.read_csv(config.TRAIN_CSV_PATH)
    print(f"\nЗагружен файл {config.TRAIN_CSV_PATH}, строк: {len(train_df_full)}")

    # Проверяем наличие нужных столбцов после полной загрузки
    if config.TRAIN_ID_COLUMN not in train_df_full.columns:
        raise KeyError(f"Столбец ID '{config.TRAIN_ID_COLUMN}' не найден в полном файле.")
    if config.TRAIN_TARGET_COLUMN is None or config.TRAIN_TARGET_COLUMN not in train_df_full.columns:
         raise KeyError(f"Столбец Target '{config.TRAIN_TARGET_COLUMN}' не найден в полном файле.")

    # Добавим полный путь к изображениям
    # Используем config.TRAIN_ID_COLUMN для получения ID
    train_df_full['image_path'] = train_df_full[config.TRAIN_ID_COLUMN].apply(
        lambda x: os.path.join(config.TRAIN_IMG_DIR, f"{x}.jpg")
    )

    # Проверка существования файлов
    print("Проверка существования файлов изображений...")
    all_files_exist = True
    for p in train_df_full['image_path'].head(5):
        if not os.path.exists(p):
            print(f"!!! Предупреждение: Файл не найден: {p}")
            all_files_exist = False
    if all_files_exist:
         print("-> Первые несколько файлов изображений найдены.")
    else:
         print("!!! Проверьте правильность путей config.DATA_PATH и config.TRAIN_IMG_DIR")

except FileNotFoundError:
    print(f"!!! Ошибка: Файл {config.TRAIN_CSV_PATH} не найден при полной загрузке.")
    train_df_full = None
except KeyError as e:
    print(f"!!! Ошибка при обработке {config.TRAIN_CSV_PATH}: {e}")
    train_df_full = None
except Exception as e:
    print(f"!!! Произошла ошибка при загрузке {config.TRAIN_CSV_PATH}: {e}")
    train_df_full = None


# Разделение на обучающую и валидационную выборки
train_df, val_df = None, None
if train_df_full is not None and config.NUM_CLASSES is not None and config.TRAIN_TARGET_COLUMN is not None:
    try:
        train_df, val_df = train_test_split(
            train_df_full,
            test_size=config.VALIDATION_SPLIT,
            random_state=config.RANDOM_STATE,
            stratify=train_df_full[config.TRAIN_TARGET_COLUMN] # Используем правильное имя столбца
        )
        print(f"\nДанные разделены: {len(train_df)} для обучения, {len(val_df)} для валидации.")
    except ValueError as e:
         print(f"!!! Ошибка при стратификации ({e}). Попытка разделения без стратификации...")
         try:
             train_df, val_df = train_test_split(
                train_df_full,
                test_size=config.VALIDATION_SPLIT,
                random_state=config.RANDOM_STATE
             )
             print(f"Данные разделены (без стратификации): {len(train_df)} для обучения, {len(val_df)} для валидации.")
         except Exception as e_split:
             print(f"!!! Не удалось разделить данные даже без стратификации: {e_split}")
    except Exception as e:
         print(f"!!! Неизвестная ошибка при разделении данных: {e}")
else:
    print("\n!!! Невозможно разделить данные: train_df_full не загружен, NUM_CLASSES не определено или TRAIN_TARGET_COLUMN не найден.")


# Определяем трансформации для изображений
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = T.Compose([
    T.Resize((config.IMAGE_SIZE, config.IMAGE_SIZE)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=15),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.05),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])

val_test_transforms = T.Compose([
    T.Resize((config.IMAGE_SIZE, config.IMAGE_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])

# Создаем кастомный класс Dataset (адаптирован под config)
class ActivityDataset(Dataset):
    # Передаем имена столбцов в конструктор
    def __init__(self, dataframe, img_dir, transform=None, is_test=False,
                 id_col='Id', labels_col='target_feature'):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        self.id_col = id_col
        self.labels_col = labels_col

        # Проверка наличия столбцов
        if self.id_col not in self.dataframe.columns:
             raise ValueError(f"Столбец ID '{self.id_col}' не найден в DataFrame для Dataset.")
        if not self.is_test and self.labels_col not in self.dataframe.columns:
             raise ValueError(f"Столбец меток '{self.labels_col}' не найден в DataFrame для Dataset (train/val).")

        # Конструируем пути к изображениям, используя правильный id_col
        self.image_paths = [os.path.join(self.img_dir, f"{img_id}.jpg")
                            for img_id in dataframe[self.id_col].values]

        if not self.is_test:
            self.labels = dataframe[self.labels_col].values

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert('RGB')
        except FileNotFoundError:
            # print(f"!!! Ошибка Dataset: Файл не найден {img_path}. Возвращаем серое.") # Можно раскомментировать для отладки
            image = Image.new('RGB', (config.IMAGE_SIZE, config.IMAGE_SIZE), color = 'grey')
        except Exception as e:
            # print(f"!!! Ошибка Dataset при открытии {img_path}: {e}. Возвращаем серое.") # Можно раскомментировать для отладки
            image = Image.new('RGB', (config.IMAGE_SIZE, config.IMAGE_SIZE), color = 'grey')

        # Получаем ID, используя правильный id_col
        img_id = self.dataframe[self.id_col].iloc[idx]

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, img_id
        else:
            label = torch.tensor(self.labels[idx], dtype=torch.long)
            return image, label

# Создаем экземпляры Dataset и DataLoader
train_loader, val_loader = None, None
if train_df is not None and val_df is not None and config.TRAIN_TARGET_COLUMN is not None:
    try:
        train_dataset = ActivityDataset(
            train_df, config.TRAIN_IMG_DIR, transform=train_transforms,
            id_col=config.TRAIN_ID_COLUMN, labels_col=config.TRAIN_TARGET_COLUMN
        )
        val_dataset = ActivityDataset(
            val_df, config.TRAIN_IMG_DIR, transform=val_test_transforms,
            id_col=config.TRAIN_ID_COLUMN, labels_col=config.TRAIN_TARGET_COLUMN
        )

        train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=False)
        val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=False)

        print(f"\nСозданы DataLoader'ы:")
        print(f" - Обучение: {len(train_dataset)} изображений, {len(train_loader)} батчей")
        print(f" - Валидация: {len(val_dataset)} изображений, {len(val_loader)} батчей")
    except ValueError as e:
        print(f"!!! Ошибка при создании Dataset (проверьте имена столбцов в DataFrame): {e}")
    except Exception as e:
        print(f"!!! Неизвестная ошибка при создании DataLoader'ов: {e}")

else:
    print("\n!!! Не удалось создать DataLoader'ы для обучения/валидации (проверьте предыдущие ошибки).")


# Подготовка данных для тестовой выборки
test_loader = None
try:
    test_image_files = [f for f in os.listdir(config.TEST_IMG_DIR) if f.endswith('.jpg')]
    if not test_image_files:
        print(f"\n!!! В папке {config.TEST_IMG_DIR} не найдены jpg файлы для теста.")
    else:
        # Предполагаем, что имя файла без расширения - это ID
        test_ids = [int(os.path.splitext(f)[0]) for f in test_image_files]
        # Создаем DataFrame с правильным именем столбца ID
        test_df = pd.DataFrame({config.TEST_ID_COLUMN: test_ids})
        test_df = test_df.sort_values(by=config.TEST_ID_COLUMN).reset_index(drop=True)

        test_dataset = ActivityDataset(
            test_df, config.TEST_IMG_DIR, transform=val_test_transforms, is_test=True,
            id_col=config.TEST_ID_COLUMN # Используем то же имя столбца для ID
        )
        test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=False)
        print(f" - Тест: {len(test_dataset)} изображений, {len(test_loader)} батчей")

except FileNotFoundError:
    print(f"\n!!! Ошибка: Папка {config.TEST_IMG_DIR} не найдена.")
except Exception as e:
    print(f"\n!!! Ошибка при подготовке тестовых данных: {e}")

In [ ]:
# Блок 4: Определение модели
# (Код остается таким же, как в предыдущем ответе)
# --------------------------
# ... (код SimpleCNN с _make_conv_block и AdaptiveAvgPool2d) ...

class SimpleCNN(nn.Module):
    def __init__(self, num_classes, image_size):
        super(SimpleCNN, self).__init__()
        self.image_size = image_size # Сохраняем размер изображения

        # --- Сверточная часть ---
        self.conv_block1 = self._make_conv_block(3, 32)      # Выход: [B, 32, H/2, W/2]
        self.conv_block2 = self._make_conv_block(32, 64)     # Выход: [B, 64, H/4, W/4]
        self.conv_block3 = self._make_conv_block(64, 128)    # Выход: [B, 128, H/8, W/8]
        self.conv_block4 = self._make_conv_block(128, 256)   # Выход: [B, 256, H/16, W/16]

        self.adaptive_pool = nn.AdaptiveAvgPool2d((6, 6)) # Приводит выход к [B, 256, 6, 6]

        # --- Полносвязная часть ---
        fc_input_features = 256 * 6 * 6

        self.fc_block = nn.Sequential(
            nn.Flatten(),
            nn.BatchNorm1d(fc_input_features),
            nn.Linear(fc_input_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.BatchNorm1d(512),
            nn.Linear(512, num_classes)
        )

    def _make_conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.adaptive_pool(x)
        x = self.fc_block(x)
        return x

# Создание экземпляра модели
model = None
if config.NUM_CLASSES is not None:
    try:
        model = SimpleCNN(num_classes=config.NUM_CLASSES, image_size=config.IMAGE_SIZE).to(config.DEVICE)
        print("\nМодель SimpleCNN создана успешно.")
        # Вывод информации о модели (опционально, требует torchinfo)
        # try:
        #     from torchinfo import summary
        #     print(summary(model, input_size=(config.BATCH_SIZE, 3, config.IMAGE_SIZE, config.IMAGE_SIZE), verbose=0))
        #     print(f"Модель перемещена на устройство: {config.DEVICE}")
        # except ImportError:
        #     print("torchinfo не найден, структура модели не выведена.")
        # except Exception as e:
        #     print(f"Ошибка при выводе summary модели: {e}")
    except Exception as e:
        print(f"!!! Ошибка при создании модели SimpleCNN: {e}")
else:
    print("\n!!! Ошибка: Количество классов (config.NUM_CLASSES) не определено. Модель не может быть создана.")

In [ ]:
# Блок 5: Обучение модели
# (Код остается таким же, как в предыдущем ответе)
# -----------------------
# ... (код цикла обучения с criterion, optimizer, scheduler, history, сохранением лучшей модели) ...

history = {'train_loss': [], 'val_loss': [], 'val_f1': []}
best_val_f1 = 0.0
best_epoch = -1

if model is not None and train_loader is not None and val_loader is not None:
    print("\n--- Настройка обучения ---")
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=3, verbose=True, min_lr=1e-6)
    print(f"Оптимизатор: Adam (lr={config.LEARNING_RATE}), Планировщик: ReduceLROnPlateau")

    print(f"\nНачинаем обучение на {config.NUM_EPOCHS} эпох...")
    training_start_time = time.time()

    for epoch in range(config.NUM_EPOCHS):
        epoch_start_time = time.time()
        model.train()
        running_loss = 0.0
        train_predictions, train_targets = [], []
        train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS} [Train]", leave=False)
        for inputs, labels in train_loop:
            inputs, labels = inputs.to(config.DEVICE), labels.to(config.DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_predictions.extend(predicted.cpu().numpy())
            train_targets.extend(labels.cpu().numpy())
            train_loop.set_postfix(loss=f"{loss.item():.4f}")

        epoch_train_loss = running_loss / len(train_dataset)
        epoch_train_f1 = f1_score(train_targets, train_predictions, average='weighted', zero_division=0)
        history['train_loss'].append(epoch_train_loss)

        model.eval()
        val_running_loss = 0.0
        val_predictions, val_targets = [], []
        with torch.no_grad():
            val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS} [Val]", leave=False)
            for inputs, labels in val_loop:
                inputs, labels = inputs.to(config.DEVICE), labels.to(config.DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_predictions.extend(predicted.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())
                val_loop.set_postfix(loss=f"{loss.item():.4f}")

        epoch_val_loss = val_running_loss / len(val_dataset)
        epoch_val_f1 = f1_score(val_targets, val_predictions, average='weighted', zero_division=0)
        history['val_loss'].append(epoch_val_loss)
        history['val_f1'].append(epoch_val_f1)
        epoch_duration = time.time() - epoch_start_time

        print(f"Epoch {epoch+1}/{config.NUM_EPOCHS} ({epoch_duration:.2f}s) - "
              f"LR: {optimizer.param_groups[0]['lr']:.1e} - "
              f"Train Loss: {epoch_train_loss:.4f}, Train F1: {epoch_train_f1:.4f} - "
              f"Val Loss: {epoch_val_loss:.4f}, Val F1: {epoch_val_f1:.4f}", end="")

        scheduler.step(epoch_val_f1)

        if epoch_val_f1 > best_val_f1:
            best_val_f1 = epoch_val_f1
            best_epoch = epoch + 1
            try:
                torch.save(model.state_dict(), config.BEST_MODEL_WEIGHTS_PATH)
                print(f" [*] New best model saved! -> {config.BEST_MODEL_WEIGHTS_PATH}")
            except Exception as e:
                print(f"!!! Error saving model: {e}")
        else:
             print(f" [ ] Val F1 did not improve from {best_val_f1:.4f} (epoch {best_epoch})")

    total_training_time = time.time() - training_start_time
    print(f"\nОбучение завершено за {total_training_time // 60:.0f} мин {total_training_time % 60:.0f} сек.")
    print(f"Лучший валидационный F1: {best_val_f1:.4f} достигнут на эпохе {best_epoch}")
else:
    print("\n!!! Обучение не может быть начато. Проверьте ошибки в предыдущих блоках.")


--- Настройка обучения ---
Оптимизатор: Adam (lr=0.001), Планировщик: ReduceLROnPlateau

Начинаем обучение на 100 эпох...


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100 [Train]:   0%|          | 0/165 [00:00<?, ?it/s]

Epoch 1/100 [Val]:   0%|          | 0/29 [00:00<?, ?it/s]

Epoch 1/100 (3181.29s) - LR: 1.0e-03 - Train Loss: 2.6228, Train F1: 0.2008 - Val Loss: 2.2711, Val F1: 0.2393 [*] New best model saved! -> /content/drive/MyDrive/kaggle/best_model_weights_20250408_074314_UTC.pth


...

Epoch 68/100 (238.56s) - LR: 1.0e-06 - Train Loss: 0.7964, Train F1: 0.7496 - Val Loss: 1.2111, Val F1: 0.6274 [ ] Val F1 did not improve from 0.6298 (epoch 63)


Epoch 69/100 [Train]:   0%|          | 0/165 [00:00<?, ?it/s]

Epoch 69/100 [Val]:   0%|          | 0/29 [00:00<?, ?it/s]

Epoch 69/100 (239.62s) - LR: 1.0e-06 - Train Loss: 0.7911, Train F1: 0.7464 - Val Loss: 1.2125, Val F1: 0.6277 [ ] Val F1 did not improve from 0.6298 (epoch 63)


Epoch 70/100 [Train]:   0%|          | 0/165 [00:00<?, ?it/s]

In [ ]:
# Блок 6: Визуализация результатов обучения
# (Код остается таким же, как в предыдущем ответе)
# -----------------------------------------
# ... (код для построения графиков Loss и F1 с использованием matplotlib) ...
if history['val_loss']:
    print("\n--- Визуализация результатов ---")
    plt.style.use('seaborn-v0_8-darkgrid')
    fig, ax1 = plt.subplots(figsize=(12, 6))
    color = 'tab:red'
    ax1.set_xlabel('Эпоха')
    ax1.set_ylabel('Loss', color=color)
    ax1.plot(range(1, len(history['train_loss']) + 1), history['train_loss'], color=color, linestyle='--', label='Train Loss')
    ax1.plot(range(1, len(history['val_loss']) + 1), history['val_loss'], color=color, label='Validation Loss')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.grid(True)
    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('F1 Score (Weighted)', color=color)
    ax2.plot(range(1, len(history['val_f1']) + 1), history['val_f1'], color=color, label='Validation F1')
    ax2.tick_params(axis='y', labelcolor=color)
    if best_epoch > 0:
        plt.axvline(x=best_epoch, color='grey', linestyle=':', label=f'Лучшая эпоха ({best_epoch})')
    fig.tight_layout()
    fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3)
    plt.title('Графики обучения: Loss и F1-score')
    plt.show()
else:
    print("\nНет данных для построения графиков.")

NameError: name 'history' is not defined

In [ ]:
# Блок 7: Предсказание на тестовых данных и создание файла для отправки
# (Код остается таким же, как в предыдущем ответе, но использует config.TEST_ID_COLUMN)
# ---------------------------------------------------------------------
# ... (код для загрузки лучшей модели, предсказания на test_loader, создания и сохранения submission_df) ...
submission_df = None

if os.path.exists(config.BEST_MODEL_WEIGHTS_PATH) and test_loader is not None and config.NUM_CLASSES is not None:
    print(f"\n--- Предсказание на тестовых данных ---")
    print(f"Загрузка лучшей модели: {config.BEST_MODEL_WEIGHTS_PATH}")
    predict_model = SimpleCNN(num_classes=config.NUM_CLASSES, image_size=config.IMAGE_SIZE).to(config.DEVICE)
    try:
        predict_model.load_state_dict(torch.load(config.BEST_MODEL_WEIGHTS_PATH, map_location=config.DEVICE))
        predict_model.eval()
        print("Веса модели загружены.")

        all_predictions = []
        all_ids = []
        print("Начинаем предсказание...")
        with torch.no_grad():
            predict_loop = tqdm(test_loader, desc="Predicting", leave=False)
            for inputs, ids in predict_loop:
                inputs = inputs.to(config.DEVICE)
                outputs = predict_model(inputs)
                _, predicted_classes = torch.max(outputs.data, 1)
                all_predictions.extend(predicted_classes.cpu().numpy())
                # Убедимся, что ids это numpy array или list
                all_ids.extend(ids.cpu().numpy() if isinstance(ids, torch.Tensor) else ids)


        print(f"Предсказание завершено. Обработано {len(all_ids)} изображений.")

        if len(all_ids) == len(all_predictions):
             # Используем config.TEST_ID_COLUMN и config.TRAIN_TARGET_COLUMN для имен столбцов
            submission_df = pd.DataFrame({
                config.TEST_ID_COLUMN: all_ids,
                config.TRAIN_TARGET_COLUMN: all_predictions # Имя колонки должно быть 'target_feature' по условию!
            })
             # ПЕРЕИМЕНУЕМ колонку с предсказаниями в 'target_feature' для сабмита!
            submission_df = submission_df.rename(columns={config.TRAIN_TARGET_COLUMN: 'target_feature'})
            # Сортируем по ID на всякий случай
            submission_df = submission_df.sort_values(by=config.TEST_ID_COLUMN).reset_index(drop=True)

            try:
                submission_df.to_csv(config.SUBMISSION_PATH, index=False)
                print(f"Файл для отправки сохранен: {config.SUBMISSION_PATH}")
                print("\nПервые 5 строк submission файла:")
                print(submission_df.head())
            except Exception as e:
                print(f"!!! Ошибка при сохранении submission файла: {e}")
        else:
            print(f"!!! Ошибка: количество ID ({len(all_ids)}) не совпадает с количеством предсказаний ({len(all_predictions)}).")

    except FileNotFoundError:
         print(f"!!! Ошибка: Файл с весами модели не найден: {config.BEST_MODEL_WEIGHTS_PATH}")
    except Exception as e:
        print(f"!!! Ошибка при загрузке модели или предсказании: {e}")

elif not os.path.exists(config.BEST_MODEL_WEIGHTS_PATH):
     print(f"\n!!! Файл лучшей модели не найден ({config.BEST_MODEL_WEIGHTS_PATH}). Предсказание невозможно.")
elif test_loader is None:
     print(f"\n!!! Тестовый DataLoader не создан. Предсказание невозможно.")
else:
     print(f"\n!!! Модель не была обучена или NUM_CLASSES не определено. Предсказание невозможно.")

# Блок для загрузки модели и сохранения csv

In [ ]:
# Блок 8: Загрузка обученной модели и предсказание (для отдельного запуска)
# =========================================================================
# Этот блок можно запустить отдельно после завершения обучения,
# при условии наличия сохраненных весов модели и тестовых данных.

import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
# Импорты стандартных библиотек и PyTorch (если запускается в новой сессии)

print("--- Запуск блока предсказания по сохраненной модели ---")

# --- 1. Настройка конфигурации (важные параметры для предсказания) ---
class PredictConfig:
    # !!! Укажите путь к папке с ТЕСТОВЫМИ изображениями !!!
    TEST_IMG_DIR = '/content/drive/MyDrive/kaggle/input/human_poses_data/img_test/' # Пример для Colab

    # !!! Укажите ТОЧНЫЙ путь к сохраненному файлу с весами модели !!!
    MODEL_WEIGHTS_PATH = '/content/drive/MyDrive/kaggle/best_model_weights_20250408_074314_UTC.pth' # <--- ЗАМЕНИТЕ НА ВАШ ФАЙЛ!

    # !!! Укажите имя файла для сохранения предсказаний !!!
    SUBMISSION_PATH = f'./submission_from_saved_{os.path.basename(MODEL_WEIGHTS_PATH)}.csv'

    # --- Параметры, которые ДОЛЖНЫ СОВПАДАТЬ с параметрами обучения ---
    IMAGE_SIZE = 128
    BATCH_SIZE = 64  # Можно увеличить для ускорения предсказания, если позволяет память GPU
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    NUM_WORKERS = 2

    # !!! Укажите КОЛИЧЕСТВО КЛАССОВ, с которым обучалась модель !!!
    NUM_CLASSES = 20  # <--- ЗАМЕНИТЕ НА ПРАВИЛЬНОЕ ЧИСЛО КЛАССОВ!

    # Имя столбца ID в тестовых данных (обычно 'Id')
    TEST_ID_COLUMN = 'Id'

predict_config = PredictConfig()

print(f"Устройство: {predict_config.DEVICE}")
print(f"Путь к тестовым изображениям: {predict_config.TEST_IMG_DIR}")
print(f"Путь к весам модели: {predict_config.MODEL_WEIGHTS_PATH}")
print(f"Количество классов: {predict_config.NUM_CLASSES}")
print(f"Путь для сохранения submission: {predict_config.SUBMISSION_PATH}")

# Проверка существования необходимых файлов/папок
if not os.path.isdir(predict_config.TEST_IMG_DIR):
    print(f"!!! Ошибка: Папка с тестовыми изображениями не найдена: {predict_config.TEST_IMG_DIR}")
    # Можно остановить выполнение, если папки нет
    # raise FileNotFoundError(f"Папка не найдена: {predict_config.TEST_IMG_DIR}")
if not os.path.exists(predict_config.MODEL_WEIGHTS_PATH):
     print(f"!!! Ошибка: Файл с весами модели не найден: {predict_config.MODEL_WEIGHTS_PATH}")
     # raise FileNotFoundError(f"Файл не найден: {predict_config.MODEL_WEIGHTS_PATH}")
if predict_config.NUM_CLASSES is None or predict_config.NUM_CLASSES <= 0:
     print(f"!!! Ошибка: Не указано корректное количество классов (NUM_CLASSES).")
     # raise ValueError("NUM_CLASSES не установлен.")

# --- 2. Определение трансформаций (должны быть как val_test_transforms при обучении) ---
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
predict_transforms = T.Compose([
    T.Resize((predict_config.IMAGE_SIZE, predict_config.IMAGE_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])
print("Трансформации для изображений определены.")

# --- 3. Определение класса Dataset (должен быть идентичен использованному при обучении) ---
class ActivityDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=True, id_col='Id'): # Упрощено для теста
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test # Всегда True в этом блоке
        self.id_col = id_col

        if self.id_col not in self.dataframe.columns:
             raise ValueError(f"Столбец ID '{self.id_col}' не найден в DataFrame для Dataset.")

        self.image_paths = [os.path.join(self.img_dir, f"{img_id}.jpg")
                            for img_id in dataframe[self.id_col].values]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception:
            # При ошибке загрузки возвращаем серое изображение и ID
            image = Image.new('RGB', (predict_config.IMAGE_SIZE, predict_config.IMAGE_SIZE), color = 'grey')
        img_id = self.dataframe[self.id_col].iloc[idx]
        if self.transform:
            image = self.transform(image)
        # Всегда возвращаем изображение и ID для предсказания
        return image, img_id
print("Класс ActivityDataset определен.")

# --- 4. Определение архитектуры модели (должна быть ИДЕНТИЧНА той, что обучалась) ---
class SimpleCNN(nn.Module):
    # КОПИЯ КЛАССА МОДЕЛИ ИЗ БЛОКА 4
    def __init__(self, num_classes, image_size):
        super(SimpleCNN, self).__init__()
        self.image_size = image_size
        self.conv_block1 = self._make_conv_block(3, 32)
        self.conv_block2 = self._make_conv_block(32, 64)
        self.conv_block3 = self._make_conv_block(64, 128)
        self.conv_block4 = self._make_conv_block(128, 256)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((6, 6))
        fc_input_features = 256 * 6 * 6
        self.fc_block = nn.Sequential(
            nn.Flatten(),
            nn.BatchNorm1d(fc_input_features),
            nn.Linear(fc_input_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5), # Dropout будет отключен в режиме eval()
            nn.BatchNorm1d(512),
            nn.Linear(512, num_classes)
        )

    def _make_conv_block(self, in_channels, out_channels):
        # Копия функции из Блока 4
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        # Копия функции из Блока 4
        x = self.conv_block1(x); x = self.conv_block2(x);
        x = self.conv_block3(x); x = self.conv_block4(x);
        x = self.adaptive_pool(x); x = self.fc_block(x);
        return x
print("Архитектура модели SimpleCNN определена.")

# --- 5. Подготовка DataLoader'а для тестовых данных ---
test_loader = None
test_df = None
try:
    test_image_files = [f for f in os.listdir(predict_config.TEST_IMG_DIR) if f.endswith('.jpg')]
    if not test_image_files:
        print(f"!!! В папке {predict_config.TEST_IMG_DIR} не найдены jpg файлы для теста.")
    else:
        test_ids = [int(os.path.splitext(f)[0]) for f in test_image_files]
        test_df = pd.DataFrame({predict_config.TEST_ID_COLUMN: test_ids})
        test_df = test_df.sort_values(by=predict_config.TEST_ID_COLUMN).reset_index(drop=True)

        test_dataset = ActivityDataset(
            test_df, predict_config.TEST_IMG_DIR, transform=predict_transforms,
            is_test=True, id_col=predict_config.TEST_ID_COLUMN
        )
        test_loader = DataLoader(
            test_dataset, batch_size=predict_config.BATCH_SIZE, shuffle=False,
            num_workers=predict_config.NUM_WORKERS, pin_memory=True
        )
        print(f"Создан тестовый DataLoader: {len(test_dataset)} изображений, {len(test_loader)} батчей.")

except FileNotFoundError:
    print(f"!!! Ошибка: Папка {predict_config.TEST_IMG_DIR} не найдена при создании DataLoader.")
except Exception as e:
    print(f"!!! Ошибка при подготовке тестового DataLoader'а: {e}")

# --- 6. Загрузка модели и предсказание ---
submission_df = None
if test_loader is not None and predict_config.NUM_CLASSES is not None and os.path.exists(predict_config.MODEL_WEIGHTS_PATH):
    try:
        print(f"\nЗагрузка весов из {predict_config.MODEL_WEIGHTS_PATH}...")
        # Создаем экземпляр модели
        model = SimpleCNN(num_classes=predict_config.NUM_CLASSES, image_size=predict_config.IMAGE_SIZE)
        # Загружаем веса
        model.load_state_dict(torch.load(predict_config.MODEL_WEIGHTS_PATH, map_location=predict_config.DEVICE))
        # Перемещаем модель на нужное устройство
        model.to(predict_config.DEVICE)
        # !!! Важно: переводим модель в режим оценки !!!
        model.eval()
        print("Модель успешно загружена и переведена в режим оценки.")

        all_predictions = []
        all_ids = []
        print("Начинаем предсказание на тестовых данных...")
        with torch.no_grad(): # Отключаем расчет градиентов
            predict_loop = tqdm(test_loader, desc="Predicting", leave=True)
            for inputs, ids in predict_loop:
                inputs = inputs.to(predict_config.DEVICE)
                outputs = model(inputs)
                _, predicted_classes = torch.max(outputs.data, 1)
                all_predictions.extend(predicted_classes.cpu().numpy())
                all_ids.extend(ids.cpu().numpy() if isinstance(ids, torch.Tensor) else ids)

        print(f"Предсказание завершено. Обработано {len(all_ids)} изображений.")

        # --- 7. Создание и сохранение файла для отправки ---
        if len(all_ids) == len(all_predictions):
            submission_df = pd.DataFrame({
                predict_config.TEST_ID_COLUMN: all_ids,
                'target_feature': all_predictions # Имя колонки по требованию соревнования
            })
            # Сортируем по ID
            submission_df = submission_df.sort_values(by=predict_config.TEST_ID_COLUMN).reset_index(drop=True)

            try:
                submission_df.to_csv(predict_config.SUBMISSION_PATH, index=False)
                print(f"\nФайл для отправки успешно сохранен: {predict_config.SUBMISSION_PATH}")
                print("\nПервые 5 строк submission файла:")
                print(submission_df.head())
            except Exception as e:
                print(f"!!! Ошибка при сохранении submission файла: {e}")
        else:
            print(f"!!! Ошибка: количество ID ({len(all_ids)}) не совпадает с количеством предсказаний ({len(all_predictions)}).")

    except FileNotFoundError:
         print(f"!!! Ошибка: Не найден файл с весами {predict_config.MODEL_WEIGHTS_PATH} при попытке загрузки.")
    except Exception as e:
        print(f"!!! Ошибка при загрузке модели или предсказании: {e}")
else:
    print("\n!!! Предсказание не может быть выполнено. Проверьте наличие тестового DataLoader'а, файла весов и корректность NUM_CLASSES.")

print("\n--- Завершение блока предсказания ---")

--- Запуск блока предсказания по сохраненной модели ---
Устройство: cpu
Путь к тестовым изображениям: /content/drive/MyDrive/kaggle/input/human_poses_data/img_test/
Путь к весам модели: /content/drive/MyDrive/kaggle/best_model_weights_20250408_074314_UTC.pth
Количество классов: 20
Путь для сохранения submission: ./submission_from_saved_best_model_weights_20250408_074314_UTC.pth.csv
Трансформации для изображений определены.
Класс ActivityDataset определен.
Архитектура модели SimpleCNN определена.
Создан тестовый DataLoader: 5301 изображений, 83 батчей.

Загрузка весов из /content/drive/MyDrive/kaggle/best_model_weights_20250408_074314_UTC.pth...
Модель успешно загружена и переведена в режим оценки.
Начинаем предсказание на тестовых данных...


Predicting:   0%|          | 0/83 [00:00<?, ?it/s]

Предсказание завершено. Обработано 5301 изображений.

Файл для отправки успешно сохранен: ./submission_from_saved_best_model_weights_20250408_074314_UTC.pth.csv

Первые 5 строк submission файла:
   Id  target_feature
0   0               0
1   3               0
2   5               2
3   8               2
4  14               0

--- Завершение блока предсказания ---


In [ ]:
# Блок 9: Исправление заголовка ID в существующем Submission файле
# ===============================================================
import pandas as pd
import os

# --- Настройки ---
# !!! Укажите ПОЛНЫЙ ПУТЬ к вашему СУЩЕСТВУЮЩЕМУ submission файлу !!!
existing_submission_path = '/content/submission_from_saved_best_model_weights_20250408_074314_UTC.pth.csv'

# Имя для НОВОГО файла с исправленным заголовком
corrected_submission_path = '/content/submission_corrected_header.csv' # Можете изменить имя

# Имя старой колонки (с большой буквы)
old_id_column = 'Id'
# Имя новой колонки (с маленькой буквы)
new_id_column = 'id'

print(f"--- Исправление файла: {existing_submission_path} ---")

try:
    # 1. Загружаем существующий CSV
    print("Загрузка существующего файла...")
    submission_df = pd.read_csv(existing_submission_path)
    print("Файл загружен. Исходные колонки:", submission_df.columns.tolist())

    # 2. Проверяем, есть ли колонка для переименования
    if old_id_column in submission_df.columns:
        # 3. Переименовываем колонку
        print(f"Переименование колонки '{old_id_column}' -> '{new_id_column}'...")
        submission_df = submission_df.rename(columns={old_id_column: new_id_column})
        print("Колонка переименована. Новые колонки:", submission_df.columns.tolist())

        # 4. Сохраняем DataFrame в новый CSV файл
        print(f"Сохранение исправленного файла в: {corrected_submission_path}...")
        submission_df.to_csv(corrected_submission_path, index=False)
        print("Новый файл успешно сохранен!")

        # 5. Выведем шапку нового файла для проверки
        print("\nПервые 5 строк ИСПРАВЛЕННОГО файла:")
        print(submission_df.head())

    else:
        print(f"!!! Ошибка: Колонка '{old_id_column}' не найдена в файле {existing_submission_path}.")
        print("Возможно, файл уже имеет правильный заголовок ('id') или колонка называется иначе.")
        print("Проверьте содержимое файла:")
        print(submission_df.head())


except FileNotFoundError:
    print(f"!!! Ошибка: Файл не найден по пути: {existing_submission_path}")
except Exception as e:
    print(f"!!! Произошла непредвиденная ошибка: {e}")

print("\n--- Завершение исправления файла ---")

--- Исправление файла: /content/submission_from_saved_best_model_weights_20250408_074314_UTC.pth.csv ---
Загрузка существующего файла...
Файл загружен. Исходные колонки: ['Id', 'target_feature']
Переименование колонки 'Id' -> 'id'...
Колонка переименована. Новые колонки: ['id', 'target_feature']
Сохранение исправленного файла в: /content/submission_corrected_header.csv...
Новый файл успешно сохранен!

Первые 5 строк ИСПРАВЛЕННОГО файла:
   id  target_feature
0   0               0
1   3               0
2   5               2
3   8               2
4  14               0

--- Завершение исправления файла ---


# Код для дообучения модели

In [ ]:
# Блок 10: Загрузка и дообучение модели
# =====================================
# Этот блок загружает сохраненные веса и продолжает обучение
# на указанное количество дополнительных эпох.

import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# Убедитесь, что все необходимые импорты присутствуют

print("--- Запуск блока дообучения модели ---")

# --- 1. Настройка конфигурации для дообучения ---
class ContinueTrainConfig:
    # --- Пути к данным (как в Блоке 2) ---
    # !!! Укажите правильный путь к вашим данным !!!
    DATA_PATH = '/content/drive/MyDrive/kaggle/input/human_poses_data/' # Пример для Colab
    TRAIN_IMG_DIR = os.path.join(DATA_PATH, 'img_train')
    # TEST_IMG_DIR не нужен для дообучения, но нужен CSV с метками
    TRAIN_CSV_PATH = os.path.join(DATA_PATH, 'train_answers.csv')
    # CATEGORIES_CSV_PATH нужен для NUM_CLASSES, если не задано вручную
    CATEGORIES_CSV_PATH = os.path.join(DATA_PATH, 'activity_categories.csv')

    # --- Пути к модели ---
    # !!! Укажите путь к файлу с весами, С КОТОРОГО ПРОДОЛЖАЕМ ОБУЧЕНИЕ !!!
    LOAD_WEIGHTS_PATH = '/content/drive/MyDrive/kaggle/best_model_weights_20250408_074314_UTC.pth' # <--- ЗАМЕНИТЕ НА ВАШ ФАЙЛ!

    # !!! Укажите путь, КУДА СОХРАНЯТЬ ЛУЧШУЮ МОДЕЛЬ ПОСЛЕ ДООБУЧЕНИЯ !!!
    # (можно перезаписать старый файл или указать новый)
    SAVE_NEW_WEIGHTS_PATH = f'/content/drive/MyDrive/kaggle/continued_best_weights_{time.strftime("%Y%m%d_%H%M%S_UTC", time.gmtime())}.pth'

    # --- Параметры модели (ДОЛЖНЫ СОВПАДАТЬ с исходным обучением) ---
    IMAGE_SIZE = 128
    # !!! Укажите КОЛИЧЕСТВО КЛАССОВ (как при исходном обучении) !!!
    NUM_CLASSES = 20 # <--- ЗАМЕНИТЕ НА ПРАВИЛЬНОЕ ЧИСЛО!

    # --- Параметры дообучения ---
    # !!! Укажите, СКОЛЬКО ЕЩЕ ЭПОХ нужно обучать !!!
    ADDITIONAL_EPOCHS = 15 # Например, еще 15 эпох

    BATCH_SIZE = 64
    # !!! РЕКОМЕНДУЕТСЯ УМЕНЬШИТЬ LEARNING RATE для дообучения !!!
    LEARNING_RATE = 1e-4 # Например, 1e-4 или 5e-5 (было 1e-3)

    VALIDATION_SPLIT = 0.15 # Должен быть тем же, что и при первом обучении, если используем тот же RANDOM_STATE
    RANDOM_STATE = 42       # Используйте тот же RANDOM_STATE для воспроизводимого разделения train/val
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    NUM_WORKERS = 2

    # Имена столбцов (как определено в Блоке 3)
    # Проверьте их правильность для вашего train_answers.csv
    TRAIN_ID_COLUMN = 'Id'
    TRAIN_TARGET_COLUMN = 'target_feature'


continue_config = ContinueTrainConfig()

print(f"Конфигурация для дообучения:")
print(f" - Загрузка весов из: {continue_config.LOAD_WEIGHTS_PATH}")
print(f" - Сохранение новых лучших весов в: {continue_config.SAVE_NEW_WEIGHTS_PATH}")
print(f" - Количество классов: {continue_config.NUM_CLASSES}") # ПРОВЕРЬТЕ ЭТО ЧИСЛО!
print(f" - Дополнительные эпохи: {continue_config.ADDITIONAL_EPOCHS}")
print(f" - Новая скорость обучения: {continue_config.LEARNING_RATE}")
print(f" - Устройство: {continue_config.DEVICE}")

# Проверка наличия файла весов
if not os.path.exists(continue_config.LOAD_WEIGHTS_PATH):
    raise FileNotFoundError(f"Файл с весами для загрузки не найден: {continue_config.LOAD_WEIGHTS_PATH}")

# Если NUM_CLASSES не задано, пробуем загрузить из CATEGORIES_CSV_PATH
if continue_config.NUM_CLASSES is None:
    try:
        activity_df = pd.read_csv(continue_config.CATEGORIES_CSV_PATH)
        if 'id' in activity_df.columns:
            continue_config.NUM_CLASSES = activity_df['id'].nunique()
            print(f"Количество классов определено из файла: {continue_config.NUM_CLASSES}")
        else:
            raise ValueError("Столбец 'id' не найден в файле категорий.")
    except Exception as e:
        raise ValueError(f"Не удалось определить NUM_CLASSES. Задайте его вручную в конфиге или проверьте CATEGORIES_CSV_PATH. Ошибка: {e}")

# Установка seed для воспроизводимости разделения данных
np.random.seed(continue_config.RANDOM_STATE)
torch.manual_seed(continue_config.RANDOM_STATE)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(continue_config.RANDOM_STATE)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# --- 2. Подготовка данных (загрузка, разделение, трансформации, даталоадеры) ---
# Этот код повторяет логику из Блока 3 для создания train_loader и val_loader

# Загрузка train_answers.csv
try:
    train_df_full = pd.read_csv(continue_config.TRAIN_CSV_PATH)
    print(f"\nЗагружен файл {continue_config.TRAIN_CSV_PATH}, строк: {len(train_df_full)}")
    # Проверка наличия нужных столбцов
    if continue_config.TRAIN_ID_COLUMN not in train_df_full.columns or continue_config.TRAIN_TARGET_COLUMN not in train_df_full.columns:
         raise KeyError(f"Не найдены столбцы '{continue_config.TRAIN_ID_COLUMN}' или '{continue_config.TRAIN_TARGET_COLUMN}' в CSV.")
    # Добавляем пути к изображениям
    train_df_full['image_path'] = train_df_full[continue_config.TRAIN_ID_COLUMN].apply(
        lambda x: os.path.join(continue_config.TRAIN_IMG_DIR, f"{x}.jpg")
    )
except Exception as e:
    raise RuntimeError(f"Ошибка при загрузке или обработке {continue_config.TRAIN_CSV_PATH}: {e}")

# Разделение на обучающую и валидационную выборки (с тем же random_state)
try:
    train_df, val_df = train_test_split(
        train_df_full,
        test_size=continue_config.VALIDATION_SPLIT,
        random_state=continue_config.RANDOM_STATE,
        stratify=train_df_full[continue_config.TRAIN_TARGET_COLUMN]
    )
    print(f"Данные разделены (с random_state={continue_config.RANDOM_STATE}): {len(train_df)} train, {len(val_df)} val.")
except Exception as e:
    raise RuntimeError(f"Ошибка при разделении данных train/val: {e}")

# Трансформации (идентичные тем, что были при обучении)
mean = [0.485, 0.456, 0.406]; std = [0.229, 0.224, 0.225]
train_transforms = T.Compose([...]) # <-- СКОПИРУЙТЕ ВАШИ train_transforms ИЗ БЛОКА 3
val_test_transforms = T.Compose([...]) # <-- СКОПИРУЙТЕ ВАШИ val_test_transforms ИЗ БЛОКА 3
# Пример (если вы использовали стандартные):
train_transforms = T.Compose([
    T.Resize((continue_config.IMAGE_SIZE, continue_config.IMAGE_SIZE)),
    T.RandomHorizontalFlip(p=0.5), T.RandomRotation(degrees=15),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.05),
    T.ToTensor(), T.Normalize(mean=mean, std=std)
])
val_test_transforms = T.Compose([
    T.Resize((continue_config.IMAGE_SIZE, continue_config.IMAGE_SIZE)),
    T.ToTensor(), T.Normalize(mean=mean, std=std)
])
print("Трансформации определены.")


# Класс Dataset (идентичный тому, что был при обучении)
class ActivityDataset(Dataset):
    # <-- СКОПИРУЙТЕ ВАШ КЛАСС ActivityDataset ИЗ БЛОКА 3 -->
    # Убедитесь, что он использует id_col и labels_col
    def __init__(self, dataframe, img_dir, transform=None, is_test=False,
                 id_col='Id', labels_col='target_feature'):
        self.dataframe = dataframe; self.img_dir = img_dir; self.transform = transform
        self.is_test = is_test; self.id_col = id_col; self.labels_col = labels_col
        if self.id_col not in self.dataframe.columns: raise ValueError(f"ID column '{self.id_col}' not found")
        if not self.is_test and self.labels_col not in self.dataframe.columns: raise ValueError(f"Label column '{self.labels_col}' not found")
        self.image_paths = [os.path.join(self.img_dir, f"{img_id}.jpg") for img_id in dataframe[self.id_col].values]
        if not self.is_test: self.labels = dataframe[self.labels_col].values
    def __len__(self): return len(self.dataframe)
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try: image = Image.open(img_path).convert('RGB')
        except Exception: image = Image.new('RGB', (continue_config.IMAGE_SIZE, continue_config.IMAGE_SIZE), color = 'grey')
        if self.transform: image = self.transform(image)
        if self.is_test: return image, self.dataframe[self.id_col].iloc[idx]
        else: return image, torch.tensor(self.labels[idx], dtype=torch.long)
print("Класс ActivityDataset определен.")

# Создание DataLoader'ов
try:
    train_dataset = ActivityDataset(
        train_df, continue_config.TRAIN_IMG_DIR, transform=train_transforms, is_test=False,
        id_col=continue_config.TRAIN_ID_COLUMN, labels_col=continue_config.TRAIN_TARGET_COLUMN
    )
    val_dataset = ActivityDataset(
        val_df, continue_config.TRAIN_IMG_DIR, transform=val_test_transforms, is_test=False,
        id_col=continue_config.TRAIN_ID_COLUMN, labels_col=continue_config.TRAIN_TARGET_COLUMN
    )
    train_loader = DataLoader(train_dataset, batch_size=continue_config.BATCH_SIZE, shuffle=True, num_workers=continue_config.NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=continue_config.BATCH_SIZE, shuffle=False, num_workers=continue_config.NUM_WORKERS, pin_memory=True)
    print(f"Созданы DataLoader'ы: {len(train_loader)} train batches, {len(val_loader)} val batches.")
except Exception as e:
    raise RuntimeError(f"Ошибка при создании DataLoader'ов: {e}")


# --- 3. Определение архитектуры модели (ИДЕНТИЧНАЯ исходной) ---
class SimpleCNN(nn.Module):
    # <-- СКОПИРУЙТЕ ВАШ КЛАСС SimpleCNN ИЗ БЛОКА 4 -->
    def __init__(self, num_classes, image_size):
        super(SimpleCNN, self).__init__(); self.image_size = image_size
        self.conv_block1 = self._make_conv_block(3, 32); self.conv_block2 = self._make_conv_block(32, 64)
        self.conv_block3 = self._make_conv_block(64, 128); self.conv_block4 = self._make_conv_block(128, 256)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((6, 6)); fc_input_features = 256 * 6 * 6
        self.fc_block = nn.Sequential(nn.Flatten(), nn.BatchNorm1d(fc_input_features),
            nn.Linear(fc_input_features, 512), nn.ReLU(inplace=True), nn.Dropout(0.5),
            nn.BatchNorm1d(512), nn.Linear(512, num_classes))
    def _make_conv_block(self, in_channels, out_channels):
         return nn.Sequential(nn.Conv2d(in_channels, out_channels, 3, 1, bias=False), nn.BatchNorm2d(out_channels), nn.ReLU(True),
                            nn.Conv2d(out_channels, out_channels, 3, 1, bias=False), nn.BatchNorm2d(out_channels), nn.ReLU(True),
                            nn.MaxPool2d(2, 2))
    def forward(self, x):
        x=self.conv_block1(x); x=self.conv_block2(x); x=self.conv_block3(x); x=self.conv_block4(x)
        x=self.adaptive_pool(x); x=self.fc_block(x); return x
print("Архитектура модели SimpleCNN определена.")


# --- 4. Загрузка модели и весов ---
try:
    model = SimpleCNN(num_classes=continue_config.NUM_CLASSES, image_size=continue_config.IMAGE_SIZE)
    print(f"\nЗагрузка весов из: {continue_config.LOAD_WEIGHTS_PATH}")
    model.load_state_dict(torch.load(continue_config.LOAD_WEIGHTS_PATH, map_location=continue_config.DEVICE))
    model.to(continue_config.DEVICE)
    print("Веса модели успешно загружены.")
except Exception as e:
    raise RuntimeError(f"Ошибка при создании или загрузке весов модели: {e}. Проверьте NUM_CLASSES и путь к файлу.")


# --- 5. Настройка для дообучения (оптимизатор, scheduler, ...) ---
criterion = nn.CrossEntropyLoss()
# Используем НОВЫЙ (уменьшенный) learning rate
optimizer = optim.Adam(model.parameters(), lr=continue_config.LEARNING_RATE, weight_decay=1e-5)
# Scheduler можно настроить так же или с другими параметрами
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=3, verbose=True, min_lr=1e-7) # Уменьшил min_lr
print(f"Настроен оптимизатор Adam с lr={continue_config.LEARNING_RATE} и ReduceLROnPlateau.")


# --- 6. Цикл дообучения (почти как в Блоке 5) ---
history = {'train_loss': [], 'val_loss': [], 'val_f1': []}
best_val_f1 = 0.0 # Начинаем отслеживать лучший F1 заново для дообучения
best_epoch = -1

print(f"\nНачинаем ДООБУЧЕНИЕ на {continue_config.ADDITIONAL_EPOCHS} эпох...")
training_start_time = time.time()

for epoch in range(continue_config.ADDITIONAL_EPOCHS):
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    train_predictions, train_targets = [], []
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{continue_config.ADDITIONAL_EPOCHS} [Continue Train]", leave=False)
    # --- Цикл обучения по батчам ---
    for inputs, labels in train_loop:
        inputs, labels = inputs.to(continue_config.DEVICE), labels.to(continue_config.DEVICE)
        optimizer.zero_grad(); outputs = model(inputs); loss = criterion(outputs, labels)
        loss.backward(); optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1); train_predictions.extend(predicted.cpu().numpy())
        train_targets.extend(labels.cpu().numpy()); train_loop.set_postfix(loss=f"{loss.item():.4f}")
    # --- Конец цикла обучения по батчам ---
    epoch_train_loss = running_loss / len(train_dataset)
    epoch_train_f1 = f1_score(train_targets, train_predictions, average='weighted', zero_division=0)
    history['train_loss'].append(epoch_train_loss)

    model.eval()
    val_running_loss = 0.0
    val_predictions, val_targets = [], []
    val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{continue_config.ADDITIONAL_EPOCHS} [Continue Val]", leave=False)
    # --- Цикл валидации по батчам ---
    with torch.no_grad():
        for inputs, labels in val_loop:
            inputs, labels = inputs.to(continue_config.DEVICE), labels.to(continue_config.DEVICE)
            outputs = model(inputs); loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1); val_predictions.extend(predicted.cpu().numpy())
            val_targets.extend(labels.cpu().numpy()); val_loop.set_postfix(loss=f"{loss.item():.4f}")
    # --- Конец цикла валидации по батчам ---
    epoch_val_loss = val_running_loss / len(val_dataset)
    epoch_val_f1 = f1_score(val_targets, val_predictions, average='weighted', zero_division=0)
    history['val_loss'].append(epoch_val_loss)
    history['val_f1'].append(epoch_val_f1)
    epoch_duration = time.time() - epoch_start_time

    print(f"Epoch {epoch+1}/{continue_config.ADDITIONAL_EPOCHS} ({epoch_duration:.2f}s) - LR: {optimizer.param_groups[0]['lr']:.1e} - "
          f"Train Loss: {epoch_train_loss:.4f}, F1: {epoch_train_f1:.4f} - Val Loss: {epoch_val_loss:.4f}, F1: {epoch_val_f1:.4f}", end="")

    scheduler.step(epoch_val_f1)

    # Сохраняем лучшую модель (используем НОВЫЙ путь)
    if epoch_val_f1 > best_val_f1:
        best_val_f1 = epoch_val_f1
        best_epoch = epoch + 1
        try:
            torch.save(model.state_dict(), continue_config.SAVE_NEW_WEIGHTS_PATH)
            print(f" [*] New best model saved! -> {continue_config.SAVE_NEW_WEIGHTS_PATH}")
        except Exception as e: print(f"!!! Error saving model: {e}")
    else: print(f" [ ] Val F1 did not improve from {best_val_f1:.4f}")

total_training_time = time.time() - training_start_time
print(f"\nДообучение завершено за {total_training_time // 60:.0f} мин {total_training_time % 60:.0f} сек.")
print(f"Лучший валидационный F1 в этой сессии дообучения: {best_val_f1:.4f}")
print(f"Лучшая модель сохранена в: {continue_config.SAVE_NEW_WEIGHTS_PATH}")

# --- 7. Опционально: Визуализация результатов ДООБУЧЕНИЯ ---
# Можно скопировать код из Блока 6 для построения графиков по данным 'history' этой сессии
if history['val_loss']:
    print("\n--- Визуализация результатов дообучения ---")
    # ... (Код построения графиков matplotlib из Блока 6) ...
    plt.style.use('seaborn-v0_8-darkgrid'); fig, ax1 = plt.subplots(figsize=(12, 6)); color = 'tab:red'
    ax1.set_xlabel(f'Эпоха дообучения (всего {continue_config.ADDITIONAL_EPOCHS})'); ax1.set_ylabel('Loss', color=color)
    ax1.plot(range(1, len(history['train_loss']) + 1), history['train_loss'], color=color, linestyle='--', label='Train Loss')
    ax1.plot(range(1, len(history['val_loss']) + 1), history['val_loss'], color=color, label='Validation Loss')
    ax1.tick_params(axis='y', labelcolor=color); ax1.grid(True); ax2 = ax1.twinx(); color = 'tab:blue'
    ax2.set_ylabel('F1 Score (Weighted)', color=color)
    ax2.plot(range(1, len(history['val_f1']) + 1), history['val_f1'], color=color, label='Validation F1')
    ax2.tick_params(axis='y', labelcolor=color);
    if best_epoch > 0: plt.axvline(x=best_epoch, color='grey', linestyle=':', label=f'Лучшая эпоха ({best_epoch})')
    fig.tight_layout(); fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3)
    plt.title('Графики дообучения: Loss и F1-score'); plt.show()


print("\n--- Завершение блока дообучения ---")